In [1]:
import pandas as pd 
import numpy as np
import plotly.graph_objects as go
import streamlit as st

## Load Data

In [2]:
df = pd.read_parquet("/Users/yiukitcheung/Documents/Projects/Stocks/train_data_repository/train_data.parquet")
df

,169EMA,BodyDiff,low,high,close_t-1,open,close_t-2,MACD_HIST,8EMA,close_t-3,...,dual_channel_Alert_1,MACD_GOLDEN_CROSS_1.0,382_Alert_1,Engulf_Alert_0,Engulf_Alert_1,CandleStickType_red,MACD_Alert_0,MACD_Alert_1,timestamp,log_daily_return
0,17.574974,0.341415,21.852482,22.496377,21.903399,22.102053,21.209587,-0.133755,21.940106,21.076820,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,NaN
1,17.627541,0.010981,21.823533,22.111040,22.443468,22.032175,21.903399,-0.117002,21.963006,21.209587,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1,-1.799747
2,17.674545,0.049914,21.288454,21.761641,22.043156,21.672793,22.443468,-0.130608,21.887422,21.903399,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,2,-1.925018
3,17.709709,0.514119,20.615608,21.382294,21.622879,21.177645,22.043156,-0.196633,21.615446,22.443468,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,3,-4.538183
4,17.742323,0.422275,20.431925,20.980983,20.663527,20.904116,21.622879,-0.241976,21.363533,22.043156,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,4,-0.883150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,64.622433,3.916343,83.373013,88.316184,82.618149,83.803941,79.663643,-0.419388,83.506737,82.409180,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,651,5.992378
652,64.894432,0.161975,85.251699,87.977240,87.720284,87.580311,82.618149,-0.043159,84.447970,79.663643,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,652,0.025080
653,65.147291,0.837857,86.285532,88.804107,87.742287,87.225369,87.720284,0.112690,84.878980,82.618149,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,653,-1.556083
654,65.357642,2.035662,81.241373,85.985580,86.387512,85.062738,87.742287,-0.006014,84.467446,87.720284,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,654,-3.967635


## Support Resistance Detection

#### Fratal Candles Patterns
##### Credit : https://towardsdatascience.com/detection-of-price-support-and-resistance-levels-in-python-baedc44c34c9
Definition: 

A fractal is a candlestick pattern made by 5 candles. The third candle has the lowest low price, the previous candles have decreasing lows and the next candles have increasing lows. By this pattern, the low of the third candle is the support level. The same concept applies to resistance levels, where the third candle has the highest high of the five ones.

![Fractals Pattern](https://miro.medium.com/v2/resize:fit:4800/format:webp/1*xOLF6WDrcvVks7CEMIVoGw.png)

### Compute Support and Resistance

* Rolling 5 days and compute the low and high
* Center Rolling Required

In [3]:
# Define the support and resistance levels
support = df[df['low'] == df['low'].rolling(5,center=True).min()][['low']]
resistance = df[df['high'] == df['high'].rolling(5,center=True).max()][['high']]


### Extract the strong Support and Resistance

* Define base on resistance/support frequency
* Set a threshold and window to define 

In [4]:
# Streamlit app
st.title("Support and Resistance Level Analysis")

# Plotting the histograms
fig = go.Figure()

# Add histogram for support level
support_hist = np.histogram(support['low'], bins=np.arange(support['low'].min(), support['low'].max() + 1, 1))
fig.add_trace(go.Histogram(
    x=support['low'], 
    name='Support Level',
    opacity=0.5,
    xbins=dict(
        start=support['low'].min(),
        end=support['low'].max(),  
        size=1
    )
))

# Add histogram for resistance level
resistance_hist = np.histogram(resistance['high'], bins=np.arange(resistance['high'].min(), resistance['high'].max() + 1, 1))
fig.add_trace(go.Histogram(
    x=resistance['high'],
    name='Resistance Level',
    opacity=0.5,
    xbins=dict(
        start=resistance['high'].min(),
        end=resistance['high'].max(),  
        size=1
    )
)) 

# Calculate the most frequent bin for support level
most_frequent_support_bin = support_hist[1][np.argmax(support_hist[0])]
most_frequent_support_count = np.max(support_hist[0])

# Calculate the most frequent bin for resistance level
most_frequent_resistance_bin = resistance_hist[1][np.argmax(resistance_hist[0])]
most_frequent_resistance_count = np.max(resistance_hist[0])

# Add bar to highlight the most frequent support bin
fig.add_trace(go.Bar(
    x=[most_frequent_support_bin],
    y=[most_frequent_support_count],
    name='Most Frequent Support',
    marker=dict(color='red', opacity=0.5)
))

# Add bar to highlight the most frequent resistance bin
fig.add_trace(go.Bar(
    x=[most_frequent_resistance_bin],
    y=[most_frequent_resistance_count],
    name='Most Frequent Resistance',
    marker=dict(color='blue', opacity=0.5)
))

# Update layout to overlay histograms
fig.update_layout(barmode='overlay')

# Display the plot in Streamlit
st.plotly_chart(fig)

2024-10-01 21:03:26.609 
  command:

    streamlit run /Users/yiukitcheung/Documents/Projects/Stocks/.venv/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

## Engulfing Method

In [5]:
# Generate random test data
np.random.seed(42)  # For reproducibility

# Define the number of rows for the test DataFrame
num_rows = 100

# Generate random dates
date_rng = pd.date_range(start='2024-09-27 09:30:00', periods=num_rows, freq='5T')

# Generate random data for the DataFrame
test_data = {
    'date': date_rng,
    'symbol': np.random.choice(['AAPL', 'GOOGL', 'MSFT', 'NVDA', 'TSLA'], size=num_rows),
    'interval': np.random.choice(['5m','15m','30m'], size=num_rows),
    '_id': [f'id_{i}' for i in range(num_rows)],
    'datetime': date_rng
}

# Generate random walk for OHLC data
initial_price = 150
price_changes = np.random.normal(loc=0, scale=1, size=num_rows)
prices = initial_price + np.cumsum(price_changes)

test_data['open'] = prices
test_data['high'] = prices + np.random.uniform(low=0, high=5, size=num_rows)
test_data['low'] = prices - np.random.uniform(low=0, high=5, size=num_rows)
test_data['close'] = prices + np.random.uniform(low=-2, high=2, size=num_rows)

# Convert to DataFrame and sort
test_data = pd.DataFrame(test_data).sort_values(by=['symbol','interval','date'])

# Display the test DataFrame
test_data

/var/folders/pv/rbjx7k1j5cs6jxs5pskztclh0000gn/T/ipykernel_64228/2835045034.py:8: FutureWarning:

'T' is deprecated and will be removed in a future version, please use 'min' instead.



,date,symbol,interval,_id,datetime,open,high,low,close
23,2024-09-27 11:25:00,AAPL,15m,id_23,2024-09-27 11:25:00,152.582965,155.239738,148.070201,152.727351
85,2024-09-27 16:35:00,AAPL,15m,id_85,2024-09-27 16:35:00,155.754880,158.482964,152.838036,153.813059
91,2024-09-27 17:05:00,AAPL,15m,id_91,2024-09-27 17:05:00,157.725543,158.072350,152.909431,157.118565
24,2024-09-27 11:30:00,AAPL,30m,id_24,2024-09-27 11:30:00,151.799712,154.502888,149.273450,151.037822
33,2024-09-27 12:15:00,AAPL,30m,id_33,2024-09-27 12:15:00,152.213762,152.606044,147.463453,151.886735
...,...,...,...,...,...,...,...,...,...
62,2024-09-27 14:40:00,TSLA,5m,id_62,2024-09-27 14:40:00,154.089648,156.547728,153.665460,155.905852
71,2024-09-27 15:25:00,TSLA,5m,id_71,2024-09-27 15:25:00,158.018970,161.148269,157.601466,159.108243
72,2024-09-27 15:30:00,TSLA,5m,id_72,2024-09-27 15:30:00,157.795507,160.311188,153.909772,157.876161
80,2024-09-27 16:10:00,TSLA,5m,id_80,2024-09-27 16:10:00,157.172253,161.873404,156.890736,158.192802


In [10]:
import pandas as pd

class CandlePattern:
    def __init__(self, candle: pd.DataFrame):
        self.candle = candle
        self.body_size = abs(self.candle['close'] - self.candle['open'])
        self.lower_shadow = min(self.candle['open'], self.candle['close']) - self.candle['low']
        self.upper_shadow = self.candle['high'] - max(self.candle['open'], self.candle['close']) 
    
    def no_volume(self):
        return self.candle['close'] == self.candle['open'] == self.candle['high'] == self.candle['low']
    
    def hammer_alert(self):
        # Hammer candle pattern: small body, long lower shadow, little or no upper shadow
        is_hammer = (
            (self.body_size <= (self.candle['high'] - self.candle['low']) * 0.3) &  # Small body
            (self.lower_shadow >= (self.candle['high'] - self.candle['low']) * 0.7) &  # Long lower shadow
            (self.upper_shadow <= (self.candle['high'] - self.candle['low']) * 0.1)  # Little or no upper shadow
        )
        
        date = pd.to_datetime(self.candle['datetime']).strftime('%Y-%m-%d %H:%M:%S')
        return is_hammer, date

    def engulf_alert(self, prev_candle: pd.DataFrame, current_candle: pd.DataFrame):
        # Previous Candle
        prev_open = prev_candle['open']
        prev_close = prev_candle['close']

        # Current Candle
        current_open = current_candle['open']
        current_close = current_candle['close']
        current_high = current_candle['high']
        
        # Bullish Engulfing: current green candle engulfs previous candle
        bullish_engulfing = (
            (current_close > current_open) &  # Current candle is green
            (current_open < min(prev_open, prev_close)) & 
            (current_close > max(prev_open, prev_close)) &
            ((current_close - current_open) > abs(prev_close - prev_open) * 1.5) &  # Current candle significantly larger
            ((current_high - current_close) * 0.7 <= (current_close - current_open)) # Little or no upper shadow
        )
        
        # Bearish Engulfing: current red candle engulfs previous candle
        bearish_engulfing = (
            (current_close < current_open) &  # Current candle is red
            (current_open > max(prev_open, prev_close)) & 
            (current_close < min(prev_open, prev_close))
        )

        date = pd.to_datetime(current_candle['datetime']).strftime('%Y-%m-%d %H:%M:%S')
        
        return bullish_engulfing, bearish_engulfing, date



In [11]:
candle = {}
last_processed_index = {}
pointer_a = 0
pointer_b = 1
alert_data = {}
last_processed_interval = 0

def streaming_process(data, interval):
    global candle, last_processed_index, pointer_a, pointer_b, alert_data, last_processed_interval

    # Store the processed index in a dict
    if data['symbol'] not in last_processed_index:
        last_processed_index[data['symbol']] = -1
        last_processed_interval = interval 
    
    # Store the candle data in a dict
    if data['symbol'] not in candle:
        candle[data['symbol']] = []
        
    candle[data['symbol']].append(data)

    # Initialize alert_data for the symbol if it does not exist
    if data['symbol'] not in alert_data:
        alert_data[data['symbol']] = []

    # If interval changes, reset the last processed index
    if last_processed_interval != interval:
        last_processed_interval = interval
        pointer_a = 0
        pointer_b = 1
        for symbol in last_processed_index:
            last_processed_index[symbol] = -1
            
    # Loop through the candle data from the last processed index
    for i in range(last_processed_index[data['symbol']] + 1, len(candle[data['symbol']])):
        if last_processed_index[data['symbol']] == -1:
            pointer_a = 0
            pointer_b = 1
            curr_candle = candle[data['symbol']][pointer_a]
            candle_pattern = CandlePattern(curr_candle)
            bullish_382, bullish_382_date = candle_pattern.hammer_alert()
            if bullish_382:
                alert_data[data['symbol']].append({'date': bullish_382_date, 'alert': 'bullish_382'})
                
            print(f"symbol: {data['symbol']} analyzing {curr_candle['datetime']} | Processed Index: {last_processed_index[data['symbol']]} | Interval: {last_processed_interval}")
            print(f"current candle: {curr_candle['datetime']}")
            
        curr_candle = candle[data['symbol']][i]
        candle_pattern = CandlePattern(curr_candle)
        # Check if there are enough candles to compare
        if len(candle[data['symbol']]) > 1 and pointer_b < len(candle[data['symbol']]):
            pre_candle = candle[data['symbol']][pointer_a]
            curr_candle = candle[data['symbol']][pointer_b]
            
            # Check for bullish 0.382 candle patterns
            bullish_382, bullish_382_date = candle_pattern.hammer_alert()

            # Check for engulfing patterns
            bullish_engulfing, bearish_engulfing, date = candle_pattern.engulf_alert(pre_candle, curr_candle)

            if bullish_engulfing:
                alert_data[data['symbol']].append({'date': date, 'alert': 'bullish_engulfing'})
                                
            # Update the last processed index
            last_processed_index[data['symbol']] = pointer_b
            print(f"symbol: {data['symbol']} comparing {pre_candle['datetime']} and {curr_candle['datetime']} | Processed Index: {last_processed_index[data['symbol']]} | Interval: {last_processed_interval}")
            print(f"pointer a: {pointer_a} | pointer b: {pointer_b}")
            print(f"current candle: {curr_candle['datetime']}")

        
        pointer_a += 1
        pointer_b += 1                

        break
    
# Test the function with the generated test data
for interval in test_data['interval'].unique():
    for index, row in test_data.iterrows():
        streaming_process(row, interval)

symbol: AAPL analyzing 2024-09-27 11:25:00 | Processed Index: -1 | Interval: 15m
current candle: 2024-09-27 11:25:00
symbol: AAPL analyzing 2024-09-27 11:25:00 | Processed Index: -1 | Interval: 15m
current candle: 2024-09-27 11:25:00
symbol: AAPL comparing 2024-09-27 11:25:00 and 2024-09-27 16:35:00 | Processed Index: 1 | Interval: 15m
pointer a: 0 | pointer b: 1
current candle: 2024-09-27 16:35:00
symbol: AAPL comparing 2024-09-27 16:35:00 and 2024-09-27 17:05:00 | Processed Index: 2 | Interval: 15m
pointer a: 1 | pointer b: 2
current candle: 2024-09-27 17:05:00
symbol: AAPL comparing 2024-09-27 17:05:00 and 2024-09-27 11:30:00 | Processed Index: 3 | Interval: 15m
pointer a: 2 | pointer b: 3
current candle: 2024-09-27 11:30:00
symbol: AAPL comparing 2024-09-27 11:30:00 and 2024-09-27 12:15:00 | Processed Index: 4 | Interval: 15m
pointer a: 3 | pointer b: 4
current candle: 2024-09-27 12:15:00
symbol: AAPL comparing 2024-09-27 12:15:00 and 2024-09-27 15:50:00 | Processed Index: 5 | Inte

In [19]:
# Fetch data from mongodb 
import pymongo
from pymongo import MongoClient
import pandas as pd

# Connect to MongoDB
client = MongoClient('localhost', 27017)
db = client['streaming_data']
collection = db['5m_stock_datastream']

# Fetch data from MongoDB
data = collection.find({'symbol': 'BSX'})
df = pd.DataFrame(list(data))
df = df.drop(columns=['_id'])
df = df[df['datetime'] >= '2024-10-01 09:30:00']
df.head()

,date,symbol,volume,datetime,low,close,open,high
78,2024-10-01 09:30:00,BSX,159020,2024-10-01 09:30:00,83.411003,83.680000,83.860001,84.000000
79,2024-10-01 09:35:00,BSX,87146,2024-10-01 09:35:00,83.614998,84.209999,83.620003,84.209999
80,2024-10-01 09:40:00,BSX,48543,2024-10-01 09:40:00,84.070000,84.129997,84.125000,84.250000
81,2024-10-01 09:45:00,BSX,44714,2024-10-01 09:45:00,83.884666,83.964996,84.099998,84.099998
82,2024-10-01 09:50:00,BSX,44518,2024-10-01 09:50:00,83.860001,83.864998,83.949997,84.059998


In [36]:
candle = {}
last_processed_index = {}
pointer_a = 0
pointer_b = 1
alert_data = {}
last_processed_interval = 0

def static_process(data):
    global candle, last_processed_index, pointer_a, pointer_b, alert_data, last_processed_interval
    global bullish_engulf_count, bearish_engulf_count, hammer_count

    # Store the processed index in a dict
    if data['symbol'] not in last_processed_index:
        last_processed_index[data['symbol']] = -1
            
    # Store the candle data in a dict
    if data['symbol'] not in candle:
        candle[data['symbol']] = []
        
    # Initialize alert_data for the symbol if it does not exist
    if data['symbol'] not in alert_data:
        alert_data[data['symbol']] = []
    
    # Append the data to the candle list
    candle[data['symbol']].append(data)
    
    # Loop through the candle data from the last processed index
    for i in range(last_processed_index[data['symbol']] + 1, len(candle[data['symbol']])):
        if last_processed_index[data['symbol']] == -1:
            pointer_a = 0
            pointer_b = 1
            curr_candle = candle[data['symbol']][pointer_a]
            candle_pattern = CandlePattern(curr_candle)
            bullish_382, bullish_382_date = candle_pattern.hammer_alert()
            if bullish_382:
                alert_data[data['symbol']].append({'date': bullish_382_date, 'alert': 'bullish_382'})
                hammer_count += 1
            
        curr_candle = candle[data['symbol']][i]                                                             
        candle_pattern = CandlePattern(curr_candle)
        
        # Check if there are enough candles to compare
        if len(candle[data['symbol']]) > 1 and pointer_b < len(candle[data['symbol']]):
            pre_candle = candle[data['symbol']][pointer_a]
            curr_candle = candle[data['symbol']][pointer_b]
            print(f"pre_candle: {pre_candle['datetime']} | curr_candle: {curr_candle['datetime']}")
            print(f"current candle: {curr_candle['datetime']}")
            # Check for bullish 0.382 candle patterns
            bullish_382, bullish_382_date = candle_pattern.hammer_alert()

            # Check for engulfing patterns
            bullish_engulfing, bearish_engulfing, date = candle_pattern.engulf_alert(pre_candle, curr_candle)

            if bullish_engulfing:
                alert_data[data['symbol']].append({'date': date, 'alert': 'bullish_engulfing'})
                bullish_engulf_count += 1
            elif bearish_engulfing:
                alert_data[data['symbol']].append({'date': date, 'alert': 'bearish_engulfing'})
                bearish_engulf_count += 1
            elif bullish_382:
                alert_data[data['symbol']].append({'date': bullish_382_date, 'alert': 'bullish_382'})
                hammer_count += 1
                
            # Update the last processed index
            last_processed_index[data['symbol']] = pointer_b
            print(f"symbol: {data['symbol']} comparing {pre_candle['datetime']} and {curr_candle['datetime']} | Processed Index: {last_processed_index[data['symbol']]}")
            print(f"pointer a: {pointer_a} | pointer b: {pointer_b}")
            print(f"current candle: {curr_candle['datetime']}")

        
        pointer_a += 1
        pointer_b += 1                

        break
    
# Test the function with the generated test data
for index, row in df.iterrows():
    static_process(row)

pre_candle: 2024-10-01 09:30:00 | curr_candle: 2024-10-01 09:35:00
current candle: 2024-10-01 09:35:00
symbol: BSX comparing 2024-10-01 09:30:00 and 2024-10-01 09:35:00 | Processed Index: 1
pointer a: 0 | pointer b: 1
current candle: 2024-10-01 09:35:00
pre_candle: 2024-10-01 09:35:00 | curr_candle: 2024-10-01 09:40:00
current candle: 2024-10-01 09:40:00
symbol: BSX comparing 2024-10-01 09:35:00 and 2024-10-01 09:40:00 | Processed Index: 2
pointer a: 1 | pointer b: 2
current candle: 2024-10-01 09:40:00
pre_candle: 2024-10-01 09:40:00 | curr_candle: 2024-10-01 09:45:00
current candle: 2024-10-01 09:45:00
symbol: BSX comparing 2024-10-01 09:40:00 and 2024-10-01 09:45:00 | Processed Index: 3
pointer a: 2 | pointer b: 3
current candle: 2024-10-01 09:45:00
pre_candle: 2024-10-01 09:45:00 | curr_candle: 2024-10-01 09:50:00
current candle: 2024-10-01 09:50:00
symbol: BSX comparing 2024-10-01 09:45:00 and 2024-10-01 09:50:00 | Processed Index: 4
pointer a: 3 | pointer b: 4
current candle: 2024

In [37]:
alert_data

{'BSX': [{'date': '2024-10-01 09:35:00', 'alert': 'bullish_engulfing'},
  {'date': '2024-10-01 11:05:00', 'alert': 'bearish_engulfing'},
  {'date': '2024-10-01 11:15:00', 'alert': 'bullish_382'},
  {'date': '2024-10-01 11:20:00', 'alert': 'bullish_engulfing'},
  {'date': '2024-10-01 11:35:00', 'alert': 'bullish_382'},
  {'date': '2024-10-01 11:55:00', 'alert': 'bullish_382'},
  {'date': '2024-10-01 12:20:00', 'alert': 'bearish_engulfing'},
  {'date': '2024-10-01 13:10:00', 'alert': 'bearish_engulfing'},
  {'date': '2024-10-01 13:20:00', 'alert': 'bearish_engulfing'},
  {'date': '2024-10-01 13:30:00', 'alert': 'bullish_382'},
  {'date': '2024-10-01 14:45:00', 'alert': 'bearish_engulfing'},
  {'date': '2024-10-01 15:00:00', 'alert': 'bearish_engulfing'},
  {'date': '2024-10-01 15:40:00', 'alert': 'bullish_382'},
  {'date': '2024-10-01 15:50:00', 'alert': 'bullish_382'},
  {'date': '2024-10-01 15:55:00', 'alert': 'bullish_382'}]}

In [22]:
# Ensure alert_data is populated
if not alert_data:
    for index, row in df.iterrows():
        static_process(row)
select_df = df[df['symbol']=='AEM']
fig = go.Figure(data=[go.Candlestick(
    x=select_df['datetime'],
    open=select_df['open'],
    high=select_df['high'],
    low=select_df['low'],
    close=select_df['close']
)])

# Annotation for the candle patterns


for alert in alert_data['AEM']:
    fig.add_annotation(
        x=alert['date'],
        y=select_df[select_df['datetime'] == alert['date']]['low'].values[0],
        xref="x",
        yref="y",
        text=alert['alert'],
        showarrow=True,
        arrowhead=1
    )
fig.update_layout(xaxis_rangeslider_visible=False)


IndexError: index 0 is out of bounds for axis 0 with size 0